# API - Extraccion de data desde Analitycs

### Documentacion de metricas y dimensiones de Google Analitycs 

[UNIVERSAL ANALITYCS](https://ga-dev-tools.web.app/dimensions-metrics-explorer/)

#### Ajustamos los parametros del notebook para visualizar todas las columnas y filas del dataframe

In [27]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#### Inicializamos las librerias que utilizaremos en la extracción

In [1]:
# -*- coding: utf-8 -*-
"""
Created on Fri Oct  1 17:27:00 2021

@author: jeff.ibacache
"""

import googleapiclient.discovery
import pprint
import time
import os, io
import argparse
import sys
import gspread
import pandas as pd
import numpy as np
import httplib2

from io import StringIO
from io import BytesIO
from googleapiclient import discovery
from oauth2client import client
from oauth2client import tools
from oauth2client.service_account import ServiceAccountCredentials
from google.oauth2 import service_account

#### Credenciales para acceder a la plataforma Google Analitycs

 1. La variable **link** va a contener la dirección de ubicaicón del archivo **.json** que contiene los permisos para acceder a la plataforma via API
 2. **VIEW_ID** va a ser el identificador para la vista de datos en la plataforma (en este caso es la vista de data jumbo web)
 
#### Identificadores correspondientes a las 4 unidades de negocio para Cenco:
1. ID: 154877638 (jumbo)
2. ID: 168222450 (Santa Isabel)
3. ID: 119926299 (Paris)
4. ID: 235051226 (Easy) 

In [3]:
link = r'C:/API Analitycs/has-latam-prod-apicm-paris-cl-a3961e1d41e0.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = link

OAUTH_SCOPES = ['https://www.googleapis.com/auth/dfareporting',
              'https://www.googleapis.com/auth/dfatrafficking',
              'https://www.googleapis.com/auth/ddmconversions',
              "https://spreadsheets.google.com/feeds",
              'https://www.googleapis.com/auth/spreadsheets',
              "https://www.googleapis.com/auth/drive.file",
              "https://www.googleapis.com/auth/drive",
              'https://www.googleapis.com/auth/analytics.readonly']

VIEW_ID = '154877638';

credentials = service_account.Credentials.from_service_account_file( link, scopes=OAUTH_SCOPES )

service = googleapiclient.discovery.build(
serviceName='analyticsreporting',
version='v4',
credentials=credentials)

#### Cuerpo del codigo de la extración de data
1. metrics y dimensions para indicar cuales serán las metricas y dimensiones que vamos a querer.
2. parametro **dateRanges** para el rango de fechas deseados para la extracción
3. parametro **includeEmptyRows** permite que la extracción traiga o no filas en blanco
4. parametro **pageSize determina** el numero de filas para una hoja de extracción.
5. en **final_data** se modifica el directorio destino donde se va a guardar la extración en formato csv

In [5]:
def handle_report(pagetoken):
    
    print('Handling Report, Page Token: ' + pagetoken)

    request = service.reports().batchGet(
        body={
          'reportRequests': [
          {
            'viewId': VIEW_ID,
            'dateRanges': [{'startDate': '2022-01-01', 'endDate': '2022-01-20'}],
            'pageToken': pagetoken,
            'pageSize': '500000',
            "includeEmptyRows": True,
            'samplingLevel': 'LARGE',      
            'metrics': [
                {'expression': 'ga:sessions'},
                {'expression': 'ga:transactions'},
                {'expression': 'ga:itemRevenue'},
                #{'expression': 'ga:users'},
                #{'expression': 'ga:dbmCost'}
            ],
            
            'dimensions': [
                #{'name': 'ga:yearWeek'},
                #{'name': 'ga:isoWeek'},
                {'name': 'ga:date'},
                {'name': 'ga:deviceCategory'},
                {'name': 'ga:source'},
                {'name': 'ga:medium'},
                {'name': 'ga:campaign'},
                #{'name': 'ga:dbmLastEventInsertionOrder'},
                #{'name': 'ga:sourceMedium'},
                
            ]
          }]
        }
    )
    
    response = request.execute()
    
    final_data = parse_data(response);
    
    final_data.rename({
        "ga:date": "Fecha",
        "ga:yearMonth": "Mes",
        "ga:campaign": "Campaña",
        "ga:medium": "Medium",
        "ga:users": "Usuarios",
        "ga:sessions": "Sesiones",
        "ga:transactions": "Transacciones",
        "ga:transactionRevenue": "Ingresos"
        }, axis=1, inplace=True)
    
    final_data.to_csv(r'C:/directorio_destino_del_archivo_extraido/data_jumbo_' + pagetoken + '.csv', index = False)

    pagetoken = response.get("reports")[0].get('nextPageToken', None)
    
    if pagetoken != None:
          handle_report(pagetoken)
            

def parse_data(response):

    reports = response['reports'][0]
    columnHeader = reports['columnHeader']['dimensions']
    metricHeader = reports['columnHeader']['metricHeader']['metricHeaderEntries']
    
    columns = columnHeader
    for metric in metricHeader:
          columns.append(metric['name'])
    
    data = pd.json_normalize(reports['data']['rows'])
    data_dimensions = pd.DataFrame(data['dimensions'].tolist())
    data_metrics = pd.DataFrame(data['metrics'].tolist())
    data_metrics = data_metrics.applymap(lambda x: x['values'])
    data_metrics = pd.DataFrame(data_metrics[0].tolist())
    result = pd.concat([data_dimensions, data_metrics], axis=1, ignore_index=True)
    result.columns = columns
    
    return result

handle_report('ga360');

Handling Report, Page Token: ga360


#### Importamos la data extraida desde la ubicación que indicamos en el cuerpo del codigo para visualizar la data en un dataframe.

1. Cambiamos el tipo de dato de las metricas a variables tipo entero con **.astype("int64")**
2. **datos.shape** para ver las dimensiones {filas x columnas}
3. funcion **head()** para ver el top 5 de las filas de la extracción

In [12]:
datos = pd.read_csv("exportaciones/data_jumbo_ga360.csv")
datos["Sesiones"] = datos["Sesiones"].astype("int64")
datos["Transacciones"] = datos["Transacciones"].astype("int64")
datos["ga:itemRevenue"] = datos["ga:itemRevenue"].astype("int64")
datos.shape

(15597, 8)

In [13]:
datos.head()

,Fecha,ga:deviceCategory,ga:source,Medium,Campaña,Sesiones,Transacciones,ga:itemRevenue
0,20220101,desktop,(direct),(none),(not set),5251,416,43103271
1,20220101,desktop,(not set),(not set),(not set),2,0,0
2,20220101,desktop,15069727_8175j,email,jumboofertas_03122021,5,2,458678
3,20220101,desktop,15074217_8176j,email,despachos_06122021,2,0,0
4,20220101,desktop,15076062_8183j,email,prime_07122021,5,0,0


In [8]:
datos[datos["Sesiones"]>=0].shape

(15597, 8)

In [9]:
datos["ga:deviceCategory"].unique()

array(['desktop', 'mobile', 'tablet'], dtype=object)

In [10]:
datos[datos["Fecha"]=="20220101"]

,Fecha,ga:deviceCategory,ga:source,Medium,Campaña,Sesiones,Transacciones,ga:itemRevenue


In [50]:
datos[(datos["Sesiones"]==0)]

,Fecha,ga:deviceCategory,ga:source,Medium,Campaña,Sesiones,Transacciones,Ingresos
2579,20220104,tablet,jumboprime.cl,referral,(not set),0,2,78850
8996,20220113,mobile,15086938_8244j,email,(not set),0,2,85643


In [9]:
datos.Canal.unique().tolist()

['(Other)',
 'Affiliate',
 'Direct',
 'Display Marketing',
 'Display Performance',
 'Email',
 'Referral',
 'SEM',
 'SEO',
 'Social',
 'Webpush']

In [10]:
datos

,Month,Canal,Usuarios,Sesiones,Transacciones,Ingresos
0,202101,(Other),137758,194330,1088,133085381
1,202101,Affiliate,154484,324086,8398,1434129727
2,202101,Direct,1117800,1783072,34724,4912555798
3,202101,Display Marketing,1166204,1988449,4858,412372825
4,202101,Display Performance,1640060,3366331,14312,1257768081
...,...,...,...,...,...,...
127,202112,Referral,504660,623787,7227,749035824
128,202112,SEM,3636758,8375709,136695,11875160953
129,202112,SEO,3357938,7055941,161799,14931245546
130,202112,Social,235677,271967,935,38740160


In [104]:
datos["Canal2"] = np.where(datos["Canal"]=="(Other)","Otros",
                   np.where(datos["Canal"]=="Direct","Directo", 
                   np.where(datos["Canal"]=="Email","Email",
                   np.where(datos["Canal"]=="Referral","Referral",
                   np.where(datos["Canal"]=="Organic Search","SEO", 
                   np.where(datos["Canal"]=="SEO","SEO",         
                   np.where(datos["Canal"]=="Social","Social Media",
                   np.where(datos["Canal"]=="Influencer Social","Social Media",           
                   np.where(datos["Canal"]=="Paid Search","Pauta de Medios",
                   np.where(datos["Canal"]=="Display","Pauta de Medios",                                      
                   np.where(datos["Canal"]=="Other Advertising","Pauta de Medios",
                   np.where(datos["Canal"]=="Branding Display","Pauta de Medios",         
                   np.where(datos["Canal"]=="Branding Search","Pauta de Medios",         
                   np.where(datos["Canal"]=="Branding Social","Pauta de Medios",         
                   np.where(datos["Canal"]=="Performance Display","Pauta de Medios",
                   np.where(datos["Canal"]=="Performance Search","Pauta de Medios",
                   np.where(datos["Canal"]=="Performance Social","Pauta de Medios",
                   np.where(datos["Canal"]=="Proveedor Display","Pauta de Medios",
                   np.where(datos["Canal"]=="Display Marketing","Pauta de Medios",
                   np.where(datos["Canal"]=="Display Performance","Pauta de Medios",
                   np.where(datos["Canal"]=="SEM","Pauta de Medios",         
                   np.where(datos["Canal"]=="Affiliate","Afiliados",
                   np.where(datos["Canal"]=="Webpush","Webpush",               
                            datos["Canal"])))))))))))))))))))))))
datos["Canal2"].unique()             

array(['Otros', 'Pauta de Medios', 'Directo', 'Email', 'Social Media',
       'SEO', 'Referral'], dtype=object)

In [105]:
df_NoPagados = datos.groupby(["Canal2"]).agg({
    "Usuarios":"sum","Sesiones":"sum","Transacciones":"sum","Ingresos":"sum"}).reset_index(drop=False)
df_NoPagados

,Canal2,Usuarios,Sesiones,Transacciones,Ingresos
0,Directo,122449,166628,16786,1494910778
1,Email,17836,30840,2390,161453737
2,Otros,17170,19342,209,14999808
3,Pauta de Medios,301444,422661,10089,776376603
4,Referral,19842,24933,666,46739171
5,SEO,171077,220951,7959,608551600
6,Social Media,73829,80908,44,2139258


#### Usamos la funcion **datos.to_excel()** para exportar y guardar nuestra data extraída en el directorio local deseado y en formato excel

In [ ]:
#datos.to_excel("exportaciones desde API/data.xlsx")